In [ ]:
import torch
import numpy as np
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg
from PIL import Image
import os

print(torch.__version__)


Folder Path

In [ ]:
folder_path = r'D:\Private\Project\kaggle\VietOCR\Recognition-Table-with-Table_Transformer-and-vietOCR\data_OCR\final_data'
label_folder = r'D:\Private\Project\kaggle\VietOCR\Recognition-Table-with-Table_Transformer-and-vietOCR\data_OCR\labels' 


Merge label


In [ ]:
import os

def merge_txt_files(folder_path, output_file):
    with open(output_file, "w", encoding="utf-8") as f_out:
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)

            if file_name.endswith(".txt") and os.path.isfile(file_path):
                with open(file_path, "r", encoding="utf-8") as f_in:
                    f_out.write(f_in.read())

    print(f"✅ Merged all .txt files into: {output_file}")

training_folder = r"D:\Private\Project\kaggle\VietOCR\Recognition-Table-with-Table_Transformer-and-vietOCR\data_OCR\labels"
output_file = r"D:\Private\Project\kaggle\VietOCR\Recognition-Table-with-Table_Transformer-and-vietOCR\data_OCR\train_label.txt"

merge_txt_files(training_folder, output_file)

Split dataset

In [ ]:
# import os
# import shutil
# from sklearn.model_selection import train_test_split


# # create train_data/old_images to split data
# data_dir = r"D:\Private\Project\kaggle\VietOCR\Recognition-Table-with-Table_Transformer-and-vietOCR\train_data\old_images"
# os.makedirs(data_dir, exist_ok=True)

# train_dir = os.path.join(data_dir, "train")
# eval_dir = os.path.join(data_dir, "eval")
# test_dir = os.path.join(data_dir, "test")
# os.makedirs(train_dir, exist_ok=True)
# os.makedirs(eval_dir, exist_ok=True)
# os.makedirs(test_dir, exist_ok=True)


# #File train
# label_file = r"D:\Private\Project\kaggle\VietOCR\Recognition-Table-with-Table_Transformer-and-vietOCR\data_OCR\train_label.txt"
# with open(label_file, "r", encoding="utf-8") as f:
#     lines = f.readlines()

# image_paths = []
# labels = []
# for line in lines:
#     image_path, label = line.strip().split("\t")
#     image_paths.append(image_path)
#     labels.append(label)

# train_paths, temp_paths, train_labels, temp_labels = train_test_split(
#     image_paths, labels, test_size=0.2, random_state=42
# )
# eval_paths, test_paths, eval_labels, test_labels = train_test_split(
#     temp_paths, temp_labels, test_size=0.5, random_state=42
# )

# def move_images_and_create_labels(image_paths, labels, target_dir, label_file):
#     with open(label_file, "w", encoding="utf-8") as f:
#         for img_path, label in zip(image_paths, labels):
#             img_name = os.path.basename(img_path)
#             new_img_path = os.path.join(target_dir, img_name)
#             shutil.copy(img_path, new_img_path)
#             abs_path = os.path.abspath(new_img_path) 
#             abs_path = abs_path.replace("\\", "/")
#             f.write(f"{abs_path}\t{label}\n")

# move_images_and_create_labels(
#     train_paths, train_labels, train_dir, os.path.join(data_dir, "rec_gt_train.txt")
# )
# move_images_and_create_labels(
#     eval_paths, eval_labels, eval_dir, os.path.join(data_dir, "rec_gt_eval.txt")
# )
# move_images_and_create_labels(
#     test_paths, test_labels, test_dir, os.path.join(data_dir, "rec_gt_test.txt")
# )

# print("Dữ liệu đã được tách thành công!")

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

base_dir = r"D:\Private\Project\kaggle\VietOCR\Recognition-Table-with-Table_Transformer-and-vietOCR"

data_dir = os.path.join(base_dir, 'train_data/old_images')
os.makedirs(data_dir, exist_ok=True)

train_dir = os.path.join(data_dir, "train")
eval_dir = os.path.join(data_dir, "eval")
test_dir = os.path.join(data_dir, "test")

for folder in [train_dir, eval_dir, test_dir]:
    os.makedirs(folder, exist_ok=True)

label_file = os.path.join(base_dir, "data_OCR/train_label.txt")

if not os.path.exists(label_file):
    raise FileNotFoundError(f"File nhãn không tồn tại: {label_file}")

image_paths = []
labels = []

with open(label_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

    for line in lines:
        try:
            image_path, label = line.strip().split("\t")

            if not os.path.isabs(image_path):
                abs_image_path = os.path.abspath(os.path.join(base_dir, image_path))
            else:
                abs_image_path = os.path.abspath(image_path)

            abs_image_path = abs_image_path.replace("\\", "/")

            if os.path.exists(abs_image_path):
                image_paths.append(abs_image_path)
                labels.append(label)
            else:
                print(f"Cảnh báo: Không tìm thấy ảnh - {abs_image_path}")
        except ValueError:
            print(f"Bỏ qua dòng lỗi định dạng: {line.strip()}")

if not image_paths:
    raise ValueError("Không có ảnh hợp lệ trong danh sách!")

train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42
)
eval_paths, test_paths, eval_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=0.5, random_state=42
)

def move_images_and_create_labels(image_paths, labels, target_dir, label_file):
    with open(label_file, "w", encoding="utf-8") as f:
        for img_path, label in zip(image_paths, labels):
            try:
                img_name = os.path.basename(img_path)
                new_img_path = os.path.join(target_dir, img_name)

                shutil.copy(img_path, new_img_path)

                abs_path = os.path.abspath(new_img_path).replace("\\", "/")

                f.write(f"{abs_path}\t{label}\n")
            except Exception as e:
                print(f"Lỗi khi di chuyển ảnh {img_path}: {e}")

move_images_and_create_labels(train_paths, train_labels, train_dir, os.path.join(data_dir, "rec_gt_train.txt"))
move_images_and_create_labels(eval_paths, eval_labels, eval_dir, os.path.join(data_dir, "rec_gt_eval.txt"))
move_images_and_create_labels(test_paths, test_labels, test_dir, os.path.join(data_dir, "rec_gt_test.txt"))

print("✅ Dữ liệu đã được tách thành công!")
print(f"📂 Số lượng ảnh train: {len(train_paths)}")
print(f"📂 Số lượng ảnh eval: {len(eval_paths)}")
print(f"📂 Số lượng ảnh test: {len(test_paths)}")

for folder, name in zip([train_dir, eval_dir, test_dir], ["train", "eval", "test"]):
    print(f"📁 Thư mục {name}: {folder}, có {len(os.listdir(folder))} ảnh")

check_img = os.path.join(train_dir, "img_280.jpg")
check_img = os.path.abspath(check_img).replace("\\", "/")
if os.path.exists(check_img):
    print(f"Ảnh tồn tại: {check_img}")
else:
    print(f"Ảnh không tồn tại: {check_img}")


Check vocab


In [ ]:
import yaml

yaml_path = r"D:\Private\Project\kaggle\VietOCR\Recognition-Table-with-Table_Transformer-and-vietOCR\config\config_after_trainer.yml"  
    config = yaml.safe_load(f)

current_vocab = config.get("vocab", "")
print("Số ký tự hiện có:", len(current_vocab))



In [ ]:
with open(yaml_path, "r", encoding="utf-8") as file:
    for line in file:
        try:
            parsed_line = yaml.safe_load(line) 
            print(parsed_line)
        except yaml.YAMLError as e:
            print(f"Lỗi khi phân tích YAML: {e}")

Fine tune

In [ ]:
from vietocr.tool.config import Cfg
from vietocr.model.trainer import Trainer

In [ ]:

config = Cfg.load_config_from_file(yaml_path)  
trainer = Trainer(config, pretrained=True)

trainer.train()


In [ ]:
!python main.py